In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [4]:
CONTEXT_SIZE=2
EMBEDDING_SIZE =10
trigrams = [([test_sentence[i],test_sentence[i+1]],test_sentence[i+2]) for i in range(len(test_sentence)-2) ]
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [5]:
vocab = set(test_sentence)
word_to_idx = {word:i for i,word in enumerate(vocab)}

In [28]:
class NGramLanguageModel(nn.Module):
    def __init__(self,vocab_size,embeding_dim,context_size):
        super(NGramLanguageModel,self).__init__()
        self.embedings = nn.Embedding(vocab_size,embeding_dim)
        self.linear1 = nn.Linear(context_size * embeding_dim,128)
        self.linear2 = nn.Linear(128,vocab_size)
    def forward(self,inputs):
        embed = self.embedings(inputs).view(1,-1)
        out=F.relu(self.linear1(embed))
        out=self.linear2(out)
        log_prod = F.log_softmax(out)
        return log_prod

losses = []
loss_fn = nn.NLLLoss()
model = NGramLanguageModel(len(vocab),EMBEDDING_SIZE,CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(),lr=0.01)
for epoch in range(20):
    total_loss = 0.
    for context ,target in trigrams:
        context_idx = torch.tensor([word_to_idx[i] for i in context],dtype=torch.long)
        model.zero_grad()
        log_prod = model(context_idx)
        loss = loss_fn(log_prod,torch.tensor([word_to_idx[target]],dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss +=loss.item()
    losses.append(total_loss)
    print('after {} epoch ,train total loss is {}'.format(epoch,total_loss))
    

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


after 0 epoch ,train total loss is 527.8221399784088
after 1 epoch ,train total loss is 502.43992161750793
after 2 epoch ,train total loss is 478.32719898223877
after 3 epoch ,train total loss is 454.71233320236206
after 4 epoch ,train total loss is 431.02923607826233
after 5 epoch ,train total loss is 406.8829126358032
after 6 epoch ,train total loss is 382.0823452472687
after 7 epoch ,train total loss is 356.49431002140045
after 8 epoch ,train total loss is 330.29604864120483
after 9 epoch ,train total loss is 303.52102506160736
after 10 epoch ,train total loss is 276.5064142346382
after 11 epoch ,train total loss is 249.6453601717949
after 12 epoch ,train total loss is 223.37196737527847
after 13 epoch ,train total loss is 198.21992456912994
after 14 epoch ,train total loss is 174.59613886475563
after 15 epoch ,train total loss is 152.85880780220032
after 16 epoch ,train total loss is 133.19035121798515
after 17 epoch ,train total loss is 115.72922551631927
after 18 epoch ,train tot

In [25]:
context_idx.size()

torch.Size([1, 2])